<a href="https://colab.research.google.com/github/ShashankHebbar006/Natural-Language-Processing/blob/main/TextClassificationUsingBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [4]:
csv_data = pd.read_csv("/content/US-Economic-News.csv",encoding = 'ISO-8859-1')
print(csv_data.shape)
checkpoint ='bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

(8000, 15)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
csv_data = csv_data[csv_data['relevance'] != 'not sure']
csv_data['relevance'] = csv_data['relevance'].map({'yes':1,'no':0})

<ipython-input-5-de941a9e87c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_data['relevance'] = csv_data['relevance'].map({'yes':1,'no':0})


In [6]:
def tokenize_func(input_data):
    return tokenizer(input_data,padding='max_length',truncation=True, return_tensors='pt')

In [7]:
train_data, tmp_data = train_test_split(csv_data,train_size=.8)
val_data, test_data = train_test_split(tmp_data,train_size=0.5)

tokenized_train_ds = train_data['text'].map(tokenize_func)
tokenized_val_ds = val_data['text'].map(tokenize_func)
tokenized_test_ds = test_data['text'].map(tokenize_func)

In [8]:
class BERTClassifier(nn.Module):
    def __init__(self,bert_model):
        super().__init__()
        self.bert = bert_model

    def fwd(self,input_data):
        op = self.bert(**input_data) # Contains only input_ids
        return op

In [9]:
m = BERTClassifier(model)
outputs = []
pred_vals = []
actual_vals = []

for i in range(tokenized_test_ds.shape[0]):
  print("------------------------------------",i,"------------------------------------")
  output = m.fwd(tokenized_test_ds[:].values[i])
  # outputs.append(output)
  print("TEXT ---> ",test_data.iloc[i]['text'])
  actual_val = test_data.iloc[i]['relevance']
  print("ACTUAL VALUE ---> ",actual_val)
  actual_vals.append(actual_val)
  pred_val = torch.argmax(output.logits.softmax(dim=1))
  print("PREDICTED VALUE ---> ",pred_val)
  pred_vals.append(pred_val)  # outputs logits take the max of logits
  print('------------------------------------------------------------------------------')

------------------------------------ 0 ------------------------------------
TEXT --->  Consumer prices edged up 0.2 percent last month as higher food and apparel costs offset lower gasoline prices, keeping inflation for the year at its lowest level since 1955, the Labor Department reported yesterday.</br></br>Council of Economic Advisers Chairman Beryl Sprinkel told reporters yesterday that the Reagan administration expects ÛÏthat the inflation rate is going to ooze up" in the next few months because of sta-</br></br>The low rate of inflation has been the brightest spot in the economy for several months, as orders for factory goods have declined, production has been sluggish and the trade deficit has repeatedly reached monthly highs.</br></br>Separately, the Commerce Department reported yesterday that new orders for durable factory goods fell 2.6 percent in August, the first drop since a 1.1 percent decline in May.</br></br>Commerce Secretary Malcolm Baldrige predicted "further progre

In [10]:
print(test_data.relevance.value_counts())
confusion_matrix(pred_vals,actual_vals)

0    676
1    124
Name: relevance, dtype: int64


array([[ 12,   6],
       [664, 118]])